In [ ]:
import praw
import pandas as pd
from datetime import datetime
import time

client_id = "AzI0ZIny0PId1YuKQ"
client_secret = "yhmWShKO5sHsgxBatvVdh1BhTg"
user_agent = "Data Extraction Script"

reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

def convert_timestamp(timestamp):
    return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M')

def scrape_subreddits_with_comments(subreddits, keyword, post_limit_per_subreddit, sleep_time=2):
    posts_data = []

    for subreddit_name in subreddits:
        subreddit = reddit.subreddit(subreddit_name)
        print(f"Searching in subreddit: {subreddit_name}...")

        for idx, submission in enumerate(subreddit.search(keyword, limit=post_limit_per_subreddit), start=1):
            post_id = submission.id
            post_datetime = convert_timestamp(submission.created_utc)
            post_content = f"Post: {submission.title}\n\n{submission.selftext}"

            submission.comment_sort = "top" 
            submission.comments.replace_more(limit=0)  
            top_comments = [comment.body for comment in submission.comments[:5]] 

            discussion_content = f"{post_content}\n\nTop Comments:\n" + "\n".join(
                [f"{i + 1}. {comment}" for i, comment in enumerate(top_comments)]
            )

            posts_data.append({
                "ID": post_id,
                "Subreddit": subreddit_name,
                "Discussion": discussion_content,
                "Datetime": post_datetime
            })

            if idx % 50 == 0:
                print(f"Processed {idx} posts from {subreddit_name}, pausing for {sleep_time} seconds...")
                time.sleep(sleep_time)

    return posts_data


subreddits_to_search = [
    "wallstreetbets",
    "stocks",
    "investing",
    "finance",
    "cryptocurrency",
    "personalfinance"]


search_keyword = "AMD"

post_limit_per_subreddit = 200 
data = scrape_subreddits_with_comments(subreddits_to_search, search_keyword, post_limit_per_subreddit)
df = pd.DataFrame(data)


output_file = "reddit_stock_keyword_posts_with_comments.csv"
df.to_csv(output_file, index=False)
print(f"Data successfully saved to {output_file}")


In [ ]:
from google.colab import files
files.download(".csv")
